In [30]:
from myMongoClient import *
from preparing_finetuning import *
import pandas as pd
%reload_ext autoreload
%autoreload 2

db = "documents-100039"
organisation_id = int(db[-6:])
mongo_client = MyMongoClient(db)

## Get a mapping form the file_name and the type of document it is

In [6]:
label_versions = pd.read_csv("data/labels_results/2024-01-18_16:15:33_df_gold_labels.csv")
output = label_versions[label_versions['label_name'] == 'document type'][['doc_id', 'label_value']]
doc_hash_value = {rows['doc_id'] : rows['label_value'] for idx , rows in output.iterrows()}
file_name_doc_hash = {inst['files'][0]['fileName'] : inst['_id'] for inst in mongo_client.get_all_documents() if inst['files'] and inst['files'][0]['fileName'].startswith(('Pseudo_',"pseudo_"))}
file_nam_value = {key : doc_hash_value[value] for key, value in file_name_doc_hash.items() if value in doc_hash_value}
# file_nam_value, len(file_nam_value)

dump(file_nam_value, "doc_type_from_file_name.pkl")

### Verify that all label name exist in the mapping label_name:label_key

In [45]:
print_all_missing_mapping()

In [74]:
all_prompts = get_all_prompts()
all_prompts

,messages,label,file_name,fields_name_key,output,doc_id
0,"[{'role': 'user', 'content': 'We are extractin...",Other,131338_13p (1-4).pdf,"[client-client storage, client-type, client-no...",{},5+ip+7sGl+kfXqh+5SwOyA==
1,"[{'role': 'user', 'content': 'We are finding i...",Type de document,131338_13p (1-4).pdf,[Type de document],{},5+ip+7sGl+kfXqh+5SwOyA==
2,"[{'role': 'user', 'content': 'We are extractin...",Other,131338_13p (5-13).pdf,"[client-client storage, client-type, client-no...",{},DQmxo4NmXgSjMB1cYnqDLQ==
3,"[{'role': 'user', 'content': 'We are finding i...",Type de document,131338_13p (5-13).pdf,[Type de document],{},DQmxo4NmXgSjMB1cYnqDLQ==
4,"[{'role': 'user', 'content': 'We are extractin...",Other,Pseudo_1002030 (11).pdf,"[client-client storage, client-type, client-no...",{},xbv5g+6M8anyCzcHvk/aAw==
...,...,...,...,...,...,...
227,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (114-114).pdf,[Date],{},qDx3ca5KQq1tPNpeTenADg==
228,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (115-116).pdf,[Date],{},TPC9ga0BcvDufyQG5rhJlA==
229,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (117-117).pdf,[Date],{},NpG1ncW32zldaK0/WnAswQ==
230,"[{'role': 'user', 'content': 'We are finding i...",Other,Pseudo_567108 (118-118).pdf,"[Date, Type de correspondance, Concerne, Signa...",{},VblBUfdotuRXo6mkyE0zJA==


In [40]:
def update_mapping_class_label_label_name(label_class, label_name):
    file = load("classifaction_label_to_label_name.pkl")
    file[label_class] = label_name
    dump(file, "classifaction_label_to_label_name.pkl") 

In [ ]:
all_prompts['fields_name_key'] = all_prompts['fields_name_key']

In [49]:
all_docs_labels = pd.read_csv("data/labels_results/2024-01-12_16:42:23_df_gold_labels.csv")
def  get_label_value(file_name, label_name):
    doc = [doc for doc in all_docs_labels if doc['files'][0]['fileName'] == file_name]
    label = [label for label in doc['labels'] if label['name'] == label_name]
    return label['value'] if label else []

all_prompts['fields_name_value'] = all_prompts.apply(lambda row : 
            {label_name : get_label_value(row['file_name'], label_name)for label_name in row['fields_name_key']}, axis = 1)

TypeError: list indices must be integers or slices, not str

In [52]:
all_docs_labels = pd.read_csv("data/labels_results/2024-01-12_16:42:23_df_gold_labels.csv")
all_docs_labels

,doc_id,label_name,label_value,confidence,model,created_on
0,0Dvjg/odKW5noqUL8wrOKw==,canton,GE,1.0,user,2024-01-12 14:33:50.439
1,0Dvjg/odKW5noqUL8wrOKw==,client,6512c03906266e001d585bb5,1.0,user,2024-01-12 14:33:50.393
2,0Dvjg/odKW5noqUL8wrOKw==,code declaration,11-345-508,1.0,user,2024-01-12 14:33:50.457
3,0Dvjg/odKW5noqUL8wrOKw==,description document,Lettre: contient généralement l'adresse de l'é...,1.0,user,2024-01-12 14:33:50.409
4,0Dvjg/odKW5noqUL8wrOKw==,document type,Déclaration fiscale,1.0,user,2024-01-12 14:33:50.403
...,...,...,...,...,...,...
508,xxNhWcwgPiDRM7jSX5dBvA==,client,6536892d127f4f001df8215e,1.0,user,2024-01-12 14:12:38.122
509,xxNhWcwgPiDRM7jSX5dBvA==,description document,Lettre: contient généralement l'addresse de l'...,1.0,user,2024-01-12 14:12:38.137
510,xxNhWcwgPiDRM7jSX5dBvA==,document type,Plan,1.0,user,2024-01-12 14:12:38.133
511,xxNhWcwgPiDRM7jSX5dBvA==,relevant date,2024-01-12,1.0,user,2024-01-12 14:12:38.146


In [69]:
user_llm = {
   "2vIzdtbIlRfELCEIj+Wsxw==" : '7wKQEgxKSZEspILT7n7UqA==',
   "4qvfu2lB0Qe9e6j0Kq+Q0Q==" : "3+h1BtgoAL0PhXWyK1+40g==",
   "HbCTvF8+qSdmSZx2554y1A==" : "VO580j3/ILBAvlhfKY7hFQ==",
   "0Dvjg/odKW5noqUL8wrOKw==" : "mHK8QQTTOTp0sx+85PF1qw==",
   "HwklrgHthb7XGEXVDgu6xg==" : "yEpfWSZXTPQZ84K+aj1drQ==",
   "HbCTvF8+qSdmSZx2554y1A==" : "VO580j3/ILBAvlhfKY7hFQ==",
   "hcqBfAlCslQ0a1QFu1RcJA==" : "63KSW0tjZ6CrwLEQoCfuUg==",
   "fTXgJLNmmgWMv9xzQjnRpQ==" : "29yradmnIewK1RIRbRSTrg==",
}

map_doc_id_doc_name = {}
all_docs = mongo_client.get_all_documents()
for doc_id in list(all_docs_labels['doc_id'].unique()):
    if doc_id in user_llm :
        doc_id = user_llm[doc_id] 
    map_doc_id_doc_name[doc_id] = [doc['files'][0]['fileName'] for doc in all_docs if doc['_id'] == doc_id and doc['files']][0]
dump(map_doc_id_doc_name, "map_doc_id_doc_name.pkl")

In [72]:
dump(map_doc_id_doc_name, "map_doc_id_doc_name.pkl")